# Extreme Weather Event Detection with ERA5 Data

This notebook demonstrates how to use the wx-extreme package to detect extreme weather events in ERA5 reanalysis data. We'll focus on detecting temperature extremes during the 2021 Pacific Northwest heat wave.

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from wx_extreme.core.detector import ExtremeEventDetector
from wx_extreme.core.evaluator import evaluate_extremes

## Load ERA5 Data

First, let's load ERA5 2m temperature data for the Pacific Northwest heat wave period in June-July 2021.

In [ ]:
def get_era5_analysis(
    fields: list[str]=['2m_temperature'],
    start_date: str="2021-06-21",
    end_date: str="2021-07-10"
) -> xr.Dataset:
    """Retrieve a subset of ERA5 data from the ARCO-ERA5 product.

    Args:
        fields: A list of field names to subset from the analysis.
        {start,end}_date: A string in "%Y-%m-%d" format bracketing the time range.
    Returns:
        An xarray.Dataset subset to the requested fields and time range.
    """
    return (
        xr.open_dataset(
            "gcs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3",
            chunks=None,
            storage_options=dict(token='anon'),
            engine='zarr'
        )
        [fields]
        .sel(
            time=slice(start_date, end_date),
            # Pacific Northwest region
            latitude=slice(42, 52),
            longitude=slice(-125, -115)
        )
    )

# Load ERA5 data
era5_ds = get_era5_analysis()
era5_ds

## Configure Event Detection

Now let's set up our extreme event detector. We'll define a temperature extreme as:
1. Temperature exceeding 35°C (95°F)
2. Persisting for at least 6 hours
3. Covering a spatial area of at least 100 km

In [ ]:
# Convert temperature from Kelvin to Celsius
era5_ds['2m_temperature'] = era5_ds['2m_temperature'] - 273.15

# Create detector
detector = ExtremeEventDetector(
    thresholds={
        '2m_temperature': {
            'threshold': 35.0,  # 35°C threshold
            'window': '6h',     # 6-hour persistence
            'spatial_scale': 100.0  # 100 km spatial scale
        }
    }
)

## Detect Extreme Events

Let's detect the extreme temperature events and evaluate their characteristics.

In [ ]:
# Detect events
events = detector.detect_events(era5_ds)

# Get event metrics
metrics = evaluate_extremes(era5_ds['2m_temperature'], detector)

print("Event Statistics:")
print(f"Number of events: {metrics['frequency']}")
print(f"Mean event duration: {metrics['mean_duration']} hours")
print(f"Maximum event duration: {metrics['max_duration']} hours")
print(f"Mean temperature during events: {metrics['mean_intensity']:.1f}°C")
print(f"Maximum temperature during events: {metrics['max_intensity']:.1f}°C")

## Visualize Events

Let's create a map showing the maximum temperature during the heat wave and areas identified as extreme events.

In [ ]:
plt.figure(figsize=(12, 8))

# Plot maximum temperature
max_temp = era5_ds['2m_temperature'].max(dim='time')
max_temp.plot(
    cmap='RdYlBu_r',
    vmin=25,
    vmax=45,
    cbar_kwargs={'label': '°C'}
)

# Overlay extreme event mask
event_mask = events['2m_temperature_extreme'].any(dim='time')
event_mask.plot.contour(
    colors='k',
    levels=[0.5],
    linewidths=2,
    label='Extreme Event Regions'
)

plt.title('Maximum Temperature and Extreme Event Regions\nduring 2021 Pacific Northwest Heat Wave')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.show()

## Time Series Analysis

Let's look at the temperature evolution at Portland, Oregon (45.5°N, 122.7°W).

In [ ]:
# Extract time series for Portland
portland_temp = era5_ds['2m_temperature'].sel(
    latitude=45.5,
    longitude=-122.7,
    method='nearest'
)

portland_events = events['2m_temperature_extreme'].sel(
    latitude=45.5,
    longitude=-122.7,
    method='nearest'
)

plt.figure(figsize=(12, 6))
portland_temp.plot(label='Temperature')

# Highlight extreme events
plt.fill_between(
    portland_temp.time,
    portland_temp.where(portland_events),
    y2=portland_temp.min(),
    alpha=0.3,
    color='red',
    label='Extreme Event'
)

plt.axhline(y=35, color='k', linestyle='--', label='Threshold (35°C)')
plt.title('Temperature Evolution at Portland, Oregon')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.grid(True)
plt.show()